# Neo4j data loading setup
Using neo4j, we’ll load in twitter data to create a network showing the relationships between legislators’ tweets, other users they mention, and hashtags they mention. 

***

In [2]:
import pandas as pd
import numpy as np
import sys
import os

In [3]:
ROOT_DIR = "../../"
DATA_DIR = ROOT_DIR + "data/"

In [4]:
# Load utils
sys.path.insert(0, os.path.abspath(ROOT_DIR + "/src"))
from utils import *

In [7]:
from neo4j import GraphDatabase
uri = 'bolt://my-neo4j:7687'
driver = GraphDatabase.driver(uri)
session = driver.session()

ValueError: Cannot resolve address my-neo4j:7687

### Create node creation constraints

In [ ]:
q = """CREATE CONSTRAINT ON (p:Person) ASSERT p.id IS UNIQUE;"""
session.run(q)
q = """CREATE CONSTRAINT ON (o:Organization) ASSERT o.id IS UNIQUE;"""
session.run(q)

***

## Data loading

In [ ]:
legislators_fname = DATA_DIR + "legislators-current.csv"
tweets_fname = DATA_DIR + "tweets.json"

### Load legislators

In [ ]:
q = """
WITH \"""" + legislators_fname + """\" AS url
LOAD CSV WITH HEADERS FROM url AS row

MERGE (p:Person {name:row.full_name, twitter:row.twitter, party:row.party, state:row.state})

RETURN row
"""
session.run(q)